In [1]:
import os
import shutil
import random
import re
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

In [2]:
# 入力ディレクトリと出力ディレクトリの設定
input_dir = Path("/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae_saka/Sakaguchi_file/S_train_and_val")
output_dir_train = Path("/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae_saka/Sakaguchi_file/S_train")
output_dir_val = Path("/mnt/nfs1/home/yamamoto-hiroto/research/vertebrae_saka/Sakaguchi_file/S_val")

# trainとvalの分割比率
val_ratio = 0.2

# 出力ディレクトリを作成
output_dir_train.mkdir(exist_ok=True)
output_dir_val.mkdir(exist_ok=True)

# 最適化されたファイル名から番号（xxxx）を抽出（正規表現使用）
number_pattern = re.compile(r'\d+')
def extract_number(filename):
    match = number_pattern.search(filename)
    return match.group() if match else ''

# ファイルを番号ごとにグループ化（最適化）
files = list(input_dir.iterdir())
groups = {}
for file_path in files:
    if file_path.is_file():
        number = extract_number(file_path.name)
        if number:
            if number not in groups:
                groups[number] = []
            groups[number].append(file_path)

# グループをシャッフルしてtrainとvalに分割
group_keys = list(groups.keys())
random.shuffle(group_keys)

val_count = int(len(group_keys) * val_ratio)
val_keys = set(group_keys[:val_count])

# 並列処理でファイルコピーを高速化
def copy_file(args):
    src_path, dest_path = args
    shutil.copy2(src_path, dest_path)  # copy2は高速でメタデータも保持
    return dest_path

# コピータスクを準備
copy_tasks = []
for number, file_paths in groups.items():
    target_dir = output_dir_val if number in val_keys else output_dir_train

    for file_path in file_paths:
        # valセットにはsegファイルを含めない
        if target_dir == output_dir_val and file_path.name.startswith("seg"):
            continue
        dest_path = target_dir / file_path.name
        copy_tasks.append((file_path, dest_path))

# 並列処理でファイルコピーを実行
with ThreadPoolExecutor(max_workers=4) as executor:
    results = list(executor.map(copy_file, copy_tasks))

print(f"trainとvalへの分割が完了しました。処理されたファイル数: {len(results)}")
print(f"valデータ: {len([k for k in val_keys])} 症例")
print(f"trainデータ: {len(group_keys) - len(val_keys)} 症例")

trainとvalへの分割が完了しました。処理されたファイル数: 114
valデータ: 6 症例
trainデータ: 24 症例
